Imports for different Packages needed for loading the autoencoder script, loading the dataset, saving weights and metrics

In [0]:
# try:
#   # %tensorflow_version only exists in Colab.
#   %tensorflow_version 2.x
# except Exception:
#   pass
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, CSVLogger, ReduceLROnPlateau
import tensorflow_datasets as tfds
import sys
import h5py
import datetime

Mounting Google Drive to import the convAE script

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
sys.path.append('/content/drive/My Drive/convAE_2_classifier')

In [0]:
from convAE import convAE

Loading the whole Dataset to train the Autoencoder on. No Validation set needed for Autoencoder

In [0]:
train, val = tfds.load(
        'cats_vs_dogs',
        split=['train[:99%]', 'train[99%:]'],
        #with_info=True,
        as_supervised=True, 
        )


In [0]:
IMG_SIZE = 256
input_shape = (IMG_SIZE, IMG_SIZE, 3)
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

Normailize the the vlaues to an interval between 0 and 1. And resize the images

In [0]:
def format_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/255.0) 
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

In [0]:
train = train.map(format_example)

Create Batches

In [0]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
train_x_batches = train_batches.map(lambda a, b: a)
train_x_batch_AE = tf.data.Dataset.zip((train_x_batches, train_x_batches))

Creating the Autoencoder

In [0]:
#tf.keras.backend.clear_session()
autoencoder = convAE(input_shape=input_shape)
autoencoder_filters = [32, 64, 128, 256]
# autoencoder_filters = [64, 128, 256, 512]
# autoencoder_filters =  [64, 128, 256, 512, 1024]

autoencoder.create_autoencoder_with_stacked_conv(filters=autoencoder_filters)

Plot autoencoder layers

In [0]:
tf.keras.utils.plot_model(autoencoder.encoder,
                          #to_file='model.png',
                          show_shapes=False,
                          show_layer_names=False,
                          #rankdir='TB',
                          expand_nested=True,
                          dpi=96
)


Get summary of Autoencoder Layers

In [0]:
autoencoder.encoder.summary()
autoencoder.decoder.summary()

Creating a file name related to the Filter sizes

In [0]:
listToStr = '-'.join(map(str, autoencoder_filters))
model_name = "convAE_stacked"
file_name = model_name + str(IMG_SIZE)+ '_' + listToStr + '_'
print(file_name

Different callbacks for saving weights, logging loss in a CSV file, reducing learning rate on plateau and earlystop training if loss doesnt get better

In [0]:
dateTime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir="./drive/My Drive/convAE_2_classifier/logs/" + file_name + "_logs_" + dateTime
weights_path="./drive/My Drive/convAE_2_classifier/weights/" + file_name + "_"  + dateTime + "_Epoch{epoch:04d}-Loss{loss:.4f}_.hdf5"
csv_path = "./drive/My Drive/convAE_2_classifier/logs/" + file_name +  "_model_history_" + dateTime + ".csv"

save_weights_callback = tf.keras.callbacks.ModelCheckpoint(filepath=weights_path , monitor='loss', save_weights_only=True, save_freq='epoch', period=5)
tensorboard_callback  = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='epoch', histogram_freq=1, write_graph=True, write_images=True)
csv_logger = CSVLogger(csv_path, separator=',', append=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=4, min_lr=0.0001, mode='min',  min_delta=0.0001, cooldown=1)
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=6, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

callbacks = [save_weights_callback, tensorboard_callback, csv_logger, reduce_lr, early_stop_callback]

Function to get learningrate metric while training

In [0]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
      return optimizer.lr 
    return lr

Compiling the Model

In [0]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9, nesterov=False)
lossfn = tf.keras.losses.MeanSquaredError()
lr_metric = get_lr_metric(optimizer)

autoencoder.autoencoder.compile(optimizer=optimizer, loss=lossfn, metrics=[ lr_metric])

Train the Model

In [0]:
history = autoencoder.autoencoder.fit(train_x_batch_AE,
                                       epochs=50,
                                       #initial_epoch=history.epoch[-1],
                                       shuffle=True,
                                       callbacks=callbacks)

Save the last Weights of the Model

In [0]:
last_loss = history.history['loss'][-1]
epoch = len(history.history['loss'])
combined = F"_Epoch{epoch:04d}-Loss{last_loss:.6f}"
autoencoder.autoencoder.save_weights("./drive/My Drive/convAE_2_classifier/weights/" + file_name + "_" + dateTime + "_" + combined +  "best.hdf5")